This example creates a couple of CAD volumes, combines them into a reactor and then performs an OpenMC / DAGMC simulation on the geometry. The geometry made is the OpenMC logo. 

There is a corresponding video fro this example
https://youtu.be/40VARwD44FA

In [ ]:
import paramak

These two code blocks create a geometry and adds relevent meta data tags that are used in the neutronics simulation.

In [ ]:
red_part = paramak.ExtrudeMixedShape(
    points=[
        (-89.0, -50.0, 'straight'),
         (-18.0, -50.0, 'straight'),
         (30.0, 30.0, 'straight'),
         (-12.0, 100.0, 'circle'),
         (88.0, -50.0, 'circle'),
    ],
    distance=10,
    color=(1,0,0),
    name='red_part',
    material_tag='red_part_material',
    stp_filename='red_part.stp',
    stl_filename='red_part.stl',
)
red_part.show()

In [ ]:
grey_part = paramak.ExtrudeMixedShape(
    points=[
         (-96.5, -27.0, 'straight'),
         (-30.0, -27.0, 'straight'),
         (3.0, 30.0, 'straight'),
         (-30.0, 87.0, 'straight'),
         (-50.0, 87.0, 'straight'),
         (-30.0, 87.0, 'circle'),
         (-97.0, 30.0, 'circle'),
    ],
    distance=10,
    color=(0.5,0.5,0.5),
    material_tag='grey_part_material',
    stp_filename='grey_part.stp',
    stl_filename='grey_part.stl',
)

grey_part.show()

This combines the two volumes into a single Reactor object

In [ ]:
both_parts = paramak.Reactor([red_part, grey_part])
both_parts.show()

This defines the neutron source used in the simulation

In [ ]:
import openmc
source = openmc.Source()
source.space = openmc.stats.Point((0, 50, 0))
source.energy = openmc.stats.Discrete([14e6], [1])
source.angle = openmc.stats.Isotropic()

This combines the geometry withthe neutron source and materials to make a model. The specifies three types of tallies (cell, mesh 2d, mesh 3d) and the reaction to tally (tritium production and heating). Output png and vtk files will be produced for the mesh tallies.

In [ ]:
my_model = paramak.NeutronicsModel(
    geometry=both_parts,
    source=source,
    materials = {'red_part_material':'P91', 'grey_part_material':'Li4SiO4'},
    mesh_tally_3d=['(n,Xt)','heating'],
    mesh_tally_2d=['(n,Xt)','heating'],
    cell_tallies=['(n,Xt)','heating', 'spectra'],
)

my_model.simulate()

This shows the cell tally results

In [ ]:
my_model.results